In [2]:
import pandas as pd
df = pd.read_csv("311_Service_Requests_2024.csv",low_memory=False)

In [3]:
df.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,63573908,12/31/2024 11:45:00 PM,12/31/2024 11:45:00 PM,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,11421,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.693663,-73.852164,"(40.69366255534651, -73.852163876474)"
1,63584208,12/31/2024 11:44:57 PM,01/01/2025 01:43:15 AM,NYPD,New York City Police Department,Illegal Fireworks,NaN,Residential Building/House,11234,2275 RYDER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.606781,-73.924298,"(40.60678061153722, -73.92429822013098)"
2,63579931,12/31/2024 11:44:57 PM,01/02/2025 05:05:54 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466,655 EAST 230 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.891872,-73.860168,"(40.89187241649303, -73.86016845296459)"
3,63577924,12/31/2024 11:44:57 PM,01/01/2025 09:01:31 AM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466,655 EAST 230 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.891872,-73.860168,"(40.89187241649303, -73.86016845296459)"
4,63578057,12/31/2024 11:44:53 PM,01/02/2025 05:08:19 PM,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10466,655 EAST 230 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.891872,-73.860168,"(40.89187241649303, -73.86016845296459)"


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458171 entries, 0 to 3458170
Data columns (total 41 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Unique Key                      int64  
 1   Created Date                    object 
 2   Closed Date                     object 
 3   Agency                          object 
 4   Agency Name                     object 
 5   Complaint Type                  object 
 6   Descriptor                      object 
 7   Location Type                   object 
 8   Incident Zip                    object 
 9   Incident Address                object 
 10  Street Name                     object 
 11  Cross Street 1                  object 
 12  Cross Street 2                  object 
 13  Intersection Street 1           object 
 14  Intersection Street 2           object 
 15  Address Type                    object 
 16  City                            object 
 17  Landmark                   

In [5]:
# Show count of nulls per column, sorted descending
df.isnull().sum().sort_values(ascending=False)

Taxi Company Borough              3455987
Road Ramp                         3449714
Bridge Highway Direction          3446628
Facility Type                     3444076
Due Date                          3438549
Bridge Highway Segment            3437034
Bridge Highway Name               3436989
Taxi Pick Up Location             3420902
Vehicle Type                      3296607
Landmark                          1369305
Intersection Street 1             1129546
Intersection Street 2             1128080
Cross Street 1                     985556
Cross Street 2                     985318
Location Type                      416849
BBL                                377892
City                               161548
Street Name                        120307
Incident Address                   120185
Descriptor                         112275
Closed Date                        103707
Resolution Description              91989
Latitude                            53599
Longitude                         

In [6]:
# Keep columns relevant to complaint analysis, resolution, geography, and agencies
columns_to_keep = [
    'Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
    'Complaint Type', 'Descriptor', 'Borough', 'Incident Zip', 'City',
    'Latitude', 'Longitude'
]
df = df[columns_to_keep]

In [7]:
df.rename(columns={
    'Unique Key': 'unique_key',
    'Created Date': 'created_date',
    'Closed Date': 'closed_date',
    'Agency': 'agency',
    'Agency Name': 'agency_name',
    'Complaint Type': 'complaint_type',
    'Descriptor': 'descriptor',
    'Borough': 'borough',
    'Incident Zip': 'incident_zip',
    'City': 'city',
    'Latitude': 'latitude',
    'Longitude': 'longitude'
}, inplace=True)

In [8]:
# Drop rows with essential missing data
df.dropna(subset=['complaint_type', 'agency', 'borough', 'created_date'], inplace=True)

# Convert datetime columns
df['created_date'] = pd.to_datetime(df['created_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
df['closed_date'] = pd.to_datetime(df['closed_date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [9]:
# Resolution time in hours
df['resolution_time_hrs'] = (df['closed_date'] - df['created_date']).dt.total_seconds() / 3600
df['weekday'] = df['created_date'].dt.day_name()
df['hour'] = df['created_date'].dt.hour

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458171 entries, 0 to 3458170
Data columns (total 15 columns):
 #   Column               Dtype         
---  ------               -----         
 0   unique_key           int64         
 1   created_date         datetime64[ns]
 2   closed_date          datetime64[ns]
 3   agency               object        
 4   agency_name          object        
 5   complaint_type       object        
 6   descriptor           object        
 7   borough              object        
 8   incident_zip         object        
 9   city                 object        
 10  latitude             float64       
 11  longitude            float64       
 12  resolution_time_hrs  float64       
 13  weekday              object        
 14  hour                 int32         
dtypes: datetime64[ns](2), float64(3), int32(1), int64(1), object(8)
memory usage: 382.6+ MB


In [11]:
df.to_csv('nyc_311_cleaned_2024.csv', index=False)